In [12]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Tag

def retrieve_page(url):
    response = requests.get(url)
    text = to_plaintext(response.text)
    text = re.sub(r'\[[0-9]*\]', '', text)
    text = re.sub(r'\[[a-z]*\]', '', text)
    return text

def to_plaintext(html_text):
    soup = BeautifulSoup(html_text, features="lxml")
    extracted_blocks = _extract_blocks(soup.body)
    extracted_blocks_texts = [block.get_text().strip() for block in extracted_blocks]
    return "\n".join(extracted_blocks_texts)

def _extract_blocks(parent_tag):
    blocks = ["p", "h1", "h2", "h3", "h4", "h5", "blockquote"] # Nomi dei tag html da cui vogliamo estrarre testo
    extracted_blocks = []
    for tag in parent_tag:
        if tag.name in blocks:
            extracted_blocks.append(tag)
            continue
        if isinstance(tag, Tag):
            if len(tag.contents) > 0:
                inner_blocks = _extract_blocks(tag)
                if len(inner_blocks) > 0:
                    extracted_blocks.extend(inner_blocks)
    return extracted_blocks

In [15]:
url = 'https://sentence.yourdictionary.com/raise'
file_path = 'sentences_raise.txt'

data = retrieve_page(url)
data = "\n".join(filter(lambda x: x.find('raise') > -1, data.split('\n')))

with open(file_path, 'a', encoding='utf8') as f:
    for sent in data:
        f.write(sent)